In [1]:
from src.learner import *
import time

from utilities import remove_cols


2024-02-13 13:00:16.236180: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-13 13:00:16.267076: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX_VNNI, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# inputs and outputs
X = remove_cols(np.genfromtxt('data/orth.csv', delimiter=","))
Y = remove_cols(np.genfromtxt('data/phon.csv', delimiter=","))

For tuning we will use a random sample of the same size that our samples will eventually be. This involves allocating 600 words for test, and the rest for train - but not using our pre-allocated samples for the purpose.

In [3]:
np.random.seed(982)

target_train_size = 300

train_n = X.shape[0]
test_n = train_n-target_train_size

sample = np.full(train_n, True, dtype=bool)

indices = np.random.choice(train_n, test_n, replace=False)

# Set chosen indices to True because they select the test items not the train items
sample[indices] = False

Limited search across HPs...

In [ ]:
seed = 387

with open('outputs/tune_1a.csv', 'w') as f:
    f.write("{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(
                                            "hidden_units",
                                            "learning_rate",
                                             "batch_size",
                                             "epochs",
                                             "loss_train",
                                             "accuracy_train",
                                             "accuracy_default_train",
                                             "mse_train",
                                             "loss_test",
                                             "accuracy_test",
                                             "accuracy_default_test",
                                             "mse_test",
                                             "time"))
    for learning_rate in [.075, .2]: 
        for batch_size in [64, 128]:
            for epochs in [50, 100, 150]:
                for hidden in [80, 100, 120]:
                    model = learner(X, Y, seed, learning_rate, hidden)
                    
                    start_time = time.time()


                    model.fit(X[sample], Y[sample], epochs=epochs, batch_size=batch_size, verbose=False)

                    end_time = time.time()
                    runtime = end_time - start_time

                    loss_train, accuracy_train, accuracy_default_train, mse_train = model.evaluate(X[sample], Y[sample], verbose=0) 
                    loss_test, accuracy_test, accuracy_default_test, mse_test = model.evaluate(X[~sample], Y[~sample], verbose=0) 

                    f.write("{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(
                                                    hidden,
                                                    learning_rate,
                                                    batch_size,
                                                    epochs,
                                                    loss_train,
                                                    accuracy_train,
                                                    accuracy_default_train,
                                                    mse_train,
                                                    loss_test,
                                                    accuracy_test,
                                                    accuracy_default_test,
                                                    mse_test,
                                                    runtime))
f.close()

In [4]:
hidden = 100
learning_rate = 0.075
batch_size = 128
epochs = 50

Let's check out item-level performance on the sample for the following configuration:

hidden = 100
learning_rate = 0.075
batch_size = 128
epochs = 50
train_accuracy = 0.990
test_accuracy = 0.961
time = 0.609

In [ ]:
seed = 387

model = learner(X, Y, seed, learning_rate, hidden)                    
model.fit(X[sample], Y[sample], epochs=epochs, batch_size=batch_size, verbose=False)

Even here, only about half of the words are perfectly correct. Let's search out further...

In [ ]:
seed = 387

with open('outputs/tune_1b.csv', 'w') as f:
    f.write("{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(
                                            "hidden_units",
                                            "learning_rate",
                                             "batch_size",
                                             "epochs",
                                             "loss_train",
                                             "accuracy_train",
                                             "accuracy_default_train",
                                             "mse_train",
                                             "loss_test",
                                             "accuracy_test",
                                             "accuracy_default_test",
                                             "mse_test",
                                             "time"))
    for learning_rate in [.2, .25]: 
        for batch_size in [64, 128]:
            for epochs in [50, 100, 150, 250, 500]:
                for hidden in [100, 150, 200]:
                    model = learner(X, Y, seed, learning_rate, hidden)
                    
                    start_time = time.time()


                    model.fit(X[sample], Y[sample], epochs=epochs, batch_size=batch_size, verbose=False)

                    end_time = time.time()
                    runtime = end_time - start_time

                    loss_train, accuracy_train, accuracy_default_train, mse_train = model.evaluate(X[sample], Y[sample], verbose=0) 
                    loss_test, accuracy_test, accuracy_default_test, mse_test = model.evaluate(X[~sample], Y[~sample], verbose=0) 

                    f.write("{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(
                                                    hidden,
                                                    learning_rate,
                                                    batch_size,
                                                    epochs,
                                                    loss_train,
                                                    accuracy_train,
                                                    accuracy_default_train,
                                                    mse_train,
                                                    loss_test,
                                                    accuracy_test,
                                                    accuracy_default_test,
                                                    mse_test,
                                                    runtime))
f.close()

Considering also the amount of compute time needed in order to learn, let's go with a learning rate of  , a batch size of  , and epochs. The train and test accuracies for this model are  and respectively. This will allow us to generate many models quickly. First, we will look around those hyperparameter spaces a bit more finely to see if we can push the accuracy a bit further.